<a href="https://colab.research.google.com/github/samuel0711/image-anomaly-detection/blob/main/con_tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import io
import sys
import os
import gc
import shutil
import numpy as np
import matplotlib.pyplot as plt
from skimage import io as skio
import torch
from torch import nn #neural networks
import torch.nn.functional as F
from torchvision.utils import save_image


import time

#Carregamento de Dados
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms



if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir("/content/drive/My Drive/Colab Notebooks")
    sys.path.append('/content/drive/My Drive/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
args = {
    'epoch':50,          #Numero de épocas
    'lr':1e-5,            #Taxa de aprendizado
    'weight_decay':5e-3,  #Penalidade L2 (Regularização)
    'batch_size': 50,     #Tamanho do batch
    'num_workers': 4
}

if torch.cuda.is_available():
  args['device'] = torch.device('cuda')
else:
  args['device'] = torch.device('cpu')

print(args['device'])


train_set = np.load('train_dataset.npy', allow_pickle = True)
test_set = np.load('test_dataset.npy', allow_pickle = True)

cuda


In [4]:
print('Treino\nMédia: ',train_set.mean(),'\t Desvio Padrão: ',train_set.std(),'\n\n')
print('Teste\nMédia: ',test_set.mean(),'\t Desvio Padrão: ',test_set.std())

Treino
Média:  105.99875310358665 	 Desvio Padrão:  48.06406962223618 


Teste
Média:  112.34485052083333 	 Desvio Padrão:  45.537924097632626


In [5]:
train_set = torch.from_numpy(train_set)

test_set = torch.from_numpy(test_set)

In [6]:
transform_train = transforms.Compose([
    transforms.Normalize((train_set.mean(), ), (train_set.std(), ))
])

transform_test = transforms.Compose([
    transforms.Normalize((test_set.mean(), ), (test_set.std(), ))
])
  
# image
train_set = transform_train(train_set)#.permute(1,2,0)
test_set = transform_test(test_set)#.permute(1,2,0)

In [78]:
train_loader = DataLoader(train_set, 
                          batch_size=args['batch_size'], 
                          shuffle=True, 
                          num_workers=args['num_workers'])

test_loader = DataLoader(test_set, 
                         batch_size=args['batch_size'], 
                         shuffle=True, 
                         num_workers=args['num_workers']) 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [91]:
#Define the Convolutional Autoencoder
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
       
        #Encoder
        self.conv1 = nn.Conv2d(1, 32, 3, padding = 1)  #in_channel, out_channel, kernel_size
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2) #kernel_size, stride
       
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.t_conv3 = nn.ConvTranspose2d(32, 1, 2, stride=2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.t_conv1(x))
        x = F.relu(self.t_conv2(x))
        x = torch.sigmoid(self.t_conv3(x))
              
        return x


#Instantiate the model
model = ConvAutoencoder()


# **Treinando**

In [92]:
from torch import optim

criterio = nn.MSELoss().to(args['device'])
optimizer = optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
model.to(args['device'])

ConvAutoencoder(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
  (t_conv1): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2))
  (t_conv2): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2))
  (t_conv3): ConvTranspose2d(32, 1, kernel_size=(3, 3), stride=(2, 2))
)

In [81]:
def save_decod_img(img, output, epoch, name):
    img = img.view(img.size(0), 1, 120, 120)
    output = output.view(output.size(0), 1, 120, 120)
    error = output.view(output.size(0), 1, 120, 120) - img.view(img.size(0), 1, 120, 120)
    save_image(img, './imgs_tcc/{}_imagem_image{}.png'.format(name,epoch))
    save_image(output, './imgs_tcc/{}_output_image{}.png'.format(name,epoch))
    save_image(error, './imgs_tcc/{}_error_image{}.png'.format(name,epoch))

## Treino ( Somente com imagens não-anomalas)

In [82]:
def train(train_loader, model, epoch, name):
  ###################
  # treinando o modelo #
  ###################
  model.train()
  
  start = time.time()

  train_loss = 0.0
  for data in train_loader:
    imagem = data.unsqueeze(1).float()
    imagem = imagem.to(args['device'])

    # clear the gradients of all optimized variables
    optimizer.zero_grad()

    # forward pass
    output = model(imagem)

    # calculando loss
    loss = criterio(output, imagem)

    # backpropagation
    loss.backward()

    # próximo passo
    optimizer.step()

    # update running training loss
    train_loss += loss.item()

  if epoch % 5 == 0:
    save_decod_img(imagem.cpu().data, output.cpu().data, epoch, name)


  train_loss = train_loss/len(train_loader)

  end = time.time()


  print('########## Train ##########')

  print("Epoca: {}\t Train Loss: {:.3f}, Time: {:.2f}\n".format(epoch, train_loss, end-start))
  #print("Medía do Loss:", epoch_loss_train.mean())

  return train_loss

## Teste ( Somente com imagens anômalas )

In [83]:
def validate(test_loader, model, epoch, name):
  ###################
  # validando o modelo #
  ###################
  model.eval()
  
  start = time.time()
  
  test_loss = 0.0
  with torch.no_grad():
    for data in test_loader:
      imagem = data.unsqueeze(1).float()
      imagem = imagem.to(args['device'])

      # forward pass
      output = model(imagem)

      # calculando loss
      loss = criterio(output, imagem)

      # update running test loss
      test_loss += loss.item()

    test_loss = test_loss/len(test_loader)
    end = time.time()

    if epoch % 5 == 0:
      save_decod_img(imagem.cpu().data, output.cpu().data, epoch, name)
    print('########## Test ##########')
    
    print("Epoca: {}\t Test Loss: {:.3f}, Time: {:.2f}\n".format(epoch, test_loss, end-start))

    return test_loss

## Rodando Teste e Treino

In [93]:
train_losses, test_losses = [], []

for epoch in range(args['epoch']):
  #Train
  train_losses.append(train(train_loader, model, epoch, 'train'))
  test_losses.append(validate(test_loader, model, epoch, 'test'))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([50, 1, 120, 120])) that is different to the input size (torch.Size([50, 1, 103, 103])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored

## Plot Treino x Teste

In [ ]:
 plt.plot(train_losses, label='Train')
 plt.plot(test_losses, label='Test')
 plt.legend(loc="upper right")
 plt.ylabel('Loss')
 plt.xlabel('Epochs')

## Plot da Imagem Original(normalizada) x Imagem Reconstruída via AutoEncoder Convolucional

In [ ]:
# obtain one batch of test images
dataiter = iter(train_loader)
images = dataiter.next()

imagem = images.unsqueeze(1).float()
imagem /= 255.0
imagem = imagem.to(args['device'])

# get sample outputs
output = model(imagem)
# prep images for display
images = images.numpy()



# output is resized into a batch of iages
output = output.view(args['batch_size'], 1, 160, 240)
# use detach when it's an output that requires_grad
output = output.detach()#.numpy()



# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(24,4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    loss = criterio(output[idx], imagem[idx])
    label = 'Loss value: {:.3f}'
    ax.set_xlabel(label.format(loss) )
    ax.set_title("Reconstructed Image")
    plt.imshow(output[idx].squeeze().cpu(), cmap='gray')
    
# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(24,4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    loss = criterio(output[idx], imagem[idx])
    label = 'Loss value: {:.3f}'
    ax.set_xlabel(label.format(loss) )
    ax.set_title("Original Image")
    plt.imshow(images[idx].squeeze(), cmap='gray')


In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images = dataiter.next()

imagem = images.unsqueeze(1).float()
imagem /= 255.0
imagem = imagem.to(args['device'])

# get sample outputs
output = model(imagem)
# prep images for display
images = images.numpy()



# output is resized into a batch of iages
output = output.view(args['batch_size'], 1, 160, 240)
# use detach when it's an output that requires_grad
output = output.detach()#.numpy()



# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(24,4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    loss = criterio(output[idx], imagem[idx])
    label = 'Loss value: {:.3f}'
    ax.set_xlabel(label.format(loss) )
    ax.set_title("Reconstructed Image")
    plt.imshow(output[idx].squeeze().cpu(), cmap='gray')
    
# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(24,4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    loss = criterio(output[idx], imagem[idx])
    label = 'Loss value: {:.3f}'
    ax.set_xlabel(label.format(loss) )
    ax.set_title("Original Image")
    plt.imshow(images[idx].squeeze(), cmap='gray')
